In [ ]:
! pip install selenium

! apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks'
! pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-upd

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np

import chromedriver_autoinstaller

import pickle

#### 크롤링 코드

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off : cloab은 새창을 지원하지않기 때문에 창 없는 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

In [ ]:
def main(pks, places):

    global driver
    driver = webdriver.Chrome(options=chrome_options)

    driver.implicitly_wait(3.5)           # 렌더링 될때까지 wait
    driver.get('https://map.kakao.com/')  # 카카오맵

    # 검색 목록
    # places = ['제주시 화북일동 베이힐']                  # 주소 + 가게명 : try - except 계속 걸어야할듯

    pk_dict = dict()
    check_place_li = list()


    try :
    # pk처리
        for pk, place in zip(pks, places):

    #for pk, place in zip(pks, places):
            temp_dict = dict()
            print(f"#####, {pk} : {place}")
            rating, num_comment, image_url, facility_li, review_dict, searched_place = crawl_info(place)
            # rating, num_comment = crawl_info(place)

            # print(f"rating : {rating}")
            # print(f"num_comment : {num_comment}")

            temp_dict["rating"] = float(rating)
            temp_dict["rating_count"] = int(num_comment)
            temp_dict["review"]   = review_dict # dict

            temp_dict["image_url"] = image_url
            temp_dict["facility"] = facility_li # list

            pk_dict[str(pk)] = temp_dict

            print(pk_dict[str(pk)])

            check_place_li.append([place, searched_place])


    # 예외처리
    except :

        print(f"error - pk:{pk}")
        return pk_dict, check_place_li


    driver.quit()

    print("finish")

    return pk_dict, check_place_li


def crawl_info(place):

    # 예외처리 : 검색불가능할 경우
    try :
        search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]')          # 카카오맵 검색창
        search_area.clear()

        # time.sleep(0.1)    # 이런거
        search_area.send_keys(place)                                                             # 검색어 입력
        driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER) # 검색

        driver.implicitly_wait(0.7 + + np.random.rand()/10)

        driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

        driver.implicitly_wait(0.4 + + np.random.rand()/10)
        time.sleep(0.25 + np.random.rand()/10)
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)  # 해당 장소
        driver.switch_to.window(driver.window_handles[-1])

        driver.implicitly_wait(3.5 + + np.random.rand()/10)

        # 파싱
        rating      = driver.find_elements(By.CSS_SELECTOR,'a[data-logevent="info_pannel,point"] > span')[0].text

        # print(rating)
        num_comment = driver.find_elements(By.CSS_SELECTOR,'a[data-logevent="info_pannel,point"] > span')[1].text[1:-1]

        # 이미지가 없는 경우가 종종있음
        if len(driver.find_elements(By.CSS_SELECTOR,'span.bg_present')) > 0 :
          image_url   = "https:"+ driver.find_element(By.CSS_SELECTOR,'span.bg_present').get_attribute("style").split('"')[1]
        else : image_url = ""

        facility_li = []
        elements = driver.find_elements(By.CSS_SELECTOR,'ul.list_facility > li > span.ico_comm')

        for e in elements :
            facility_li.append(e.get_attribute("class").split()[-1].split("_")[-1])

        # get_review 함수는 다시 짜기
        review_dict = get_review()

        searched_place = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.1 + np.random.rand()/10)


    except :


        # 후기미제공 등 처리
        if len(driver.find_elements(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')) > 0 :
            searched_place = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text

            facility_li = []
            elements_f = driver.find_elements(By.CSS_SELECTOR,'ul.list_facility > li > span.ico_comm')
            if len(elements_f) > 0  :
                for e in elements_f :
                    facility_li.append(e.get_attribute("class").split()[-1].split("_")[-1])

            if len(driver.find_elements(By.CSS_SELECTOR,'span.bg_present')) > 0 :
              image_url   = "https:"+ driver.find_element(By.CSS_SELECTOR,'span.bg_present').get_attribute("style").split('"')[1]
            else : image_url =""

            # rating      = driver.find_elements(By.CSS_SELECTOR,'a[data-logevent="info_pannel,point"] > span')[0].text
            if len(driver.find_elements(By.CSS_SELECTOR,'span.txt_blind')) > 0 :
              rating = -1

            # 예외처리 추가
            else : rating = 0

            driver.close() # 여기서는 필요함

        else :

            searched_place = ""
            facility_li = []
            image_url = ""
            rating = 0

        num_comment = 0
        review_dict = {}


        # 필요할듯
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.1 + np.random.rand()/10)


    return rating, num_comment, image_url, facility_li, review_dict, searched_place


def get_review() :
    """
    예외처리 때문에 함수를 하나 만드는게 유리할듯
    """

    review_dict = dict()
    i = 1   # init

    # 처음 3개
    review_li = driver.find_elements(By.CSS_SELECTOR, "ul.list_evaluation > li")

    if len(review_li) != 0:
        for review in review_li:
            user_name = review.find_element(By.CSS_SELECTOR, "span.txt_username").text  # 유저명
            review = review.find_element(By.CSS_SELECTOR, "p.txt_comment > span").text  # 리뷰

            review_temp_dict = dict()
            # review 필터링
            if len(review) > 0 :
                review_temp_dict["user_name"] = user_name
                review_temp_dict["review"] = review

                review_dict[str(i)] = review_temp_dict
                i += 1

    exiting_len = len(review_li) # 3


    try :
        while True :

            if driver.find_element(By.CSS_SELECTOR, "span.txt_more").text == "후기 더보기" :

                # 후기 더보기 - 클릭
                btn = driver.find_element(By.CSS_SELECTOR, 'span.txt_more')
                driver.execute_script("arguments[0].click();", btn)

                driver.implicitly_wait(0.2 + np.random.rand()/10)
                time.sleep(0.25 + np.random.rand()/10)
                review_li = driver.find_elements(By.CSS_SELECTOR, "ul.list_evaluation > li")
                new_len = len(review_li) # 8
                check_len = new_len - exiting_len
                exiting_len = len(review_li) # update

                # print(f"{new_len}, {check_len}, {exiting_len}")


                review_li = review_li[-(check_len):]

                if len(review_li) != 0:
                    for review in review_li:
                        user_name = review.find_element(By.CSS_SELECTOR, "span.txt_username").text  # 유저명
                        review = review.find_element(By.CSS_SELECTOR, "p.txt_comment > span").text  # 리뷰

                        review_temp_dict = dict()
                        # review 필터링
                        if len(review) > 0 :
                            review_temp_dict["user_name"] = user_name
                            review_temp_dict["review"] = review

                            review_dict[str(i)] = review_temp_dict
                            i += 1
                            # print(f"i {i}")
                            if i > 100 :
                              break  # 더 집어넣아야함
                if i > 100 :
                  break

            else : break


    except :     return review_dict


    return review_dict


#### 크롤링 대상

In [ ]:
%cd  /content/drive/MyDrive/RAG

/content/drive/MyDrive/RAG


In [ ]:
df = pd.read_csv("unique_mct.csv", encoding="cp949")
df = df[["pk", "MCT_NM", "OP_YMD", "ADDR"]] # 순서 재정렬

# ['(사)', '(유)', '(주)', '(德)', 또는 '(제주)')] 제거 후 > 두칸 공백 제거
df['MCT_NM_1'] = df['MCT_NM'].replace(to_replace=r'\(.{1,2}\)', value='', regex=True).replace(to_replace=r'\s{2}', value=' ', regex=True)

# df[df["ADDR"].str.extract(r'([동])')[0].isna()]
# 동 또는 면/읍까지 활용, 앞 제주는 버림
df["ADDR_1"] = df["ADDR"].apply(lambda x : " ".join(x.split()[1:3]))

df["search_1"] = (df["ADDR_1"] + " "+ df["MCT_NM_1"]).replace(to_replace=r'\s{2}', value=' ', regex=True)

In [ ]:
place_li = df["search_1"].iloc[8501:].to_list()
pk_li = df["pk"].iloc[8501:].to_list()

In [ ]:
df["pk"].iloc[8501:]

,pk
8501,8501
8502,8502
8503,8503
8504,8504
8505,8505
...,...
9501,9501
9502,9502
9503,9503
9504,9504


In [ ]:
len(place_li), len(pk_li)

(1005, 1005)

In [ ]:
# check

# total_dict, check = main([3757], ["서귀포시 동홍동 사리원식당"])

In [ ]:
total_dict, check = main(pk_li, place_li)

with open('dict_8501_.pickle','wb') as fw:

    pickle.dump(total_dict, fw)

with open('check_8501_.pickle','wb') as fw:

    pickle.dump(check, fw)

#####, 8501 : 제주시 한경면 판포리아
{'rating': 0.0, 'rating_count': 0, 'review': {}, 'image_url': '', 'facility': []}
#####, 8502 : 제주시 한경면 판포미인
{'rating': -1.0, 'rating_count': 0, 'review': {}, 'image_url': 'https://t1.kakaocdn.net/thumb/T800x0.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocalfiy%2F4026A0CE48A04F80BDF0778B4C29363E', 'facility': ['wifi', 'noanimal', 'parking', 'nohandicapped', 'noplayroom', 'nosmoking']}
#####, 8503 : 제주시 이도이동 팔각촌
{'rating': 4.6, 'rating_count': 12, 'review': {'1': {'user_name': '맛도리', 'review': '한라산 산행후 먹었는데 개꿀맛 ㅠ\n숄더랙 목살 항정살 먹었는데\n목살 개꿀마탱'}, '2': {'user_name': '시현', 'review': '여긴 고기도 고기지만 라면이 기가막혀요~\n술이 술술 들어감'}, '3': {'user_name': '🌴(몰아서 쓰기도 함)', 'review': '백돼지근고기 초벌해서 갖다주시는데 시간 좀 걸림\n맛있음\n의자에 등받이 없어서 불편\n달걀찜도 괜찮고 국수 시원하니 괜찮음'}, '4': {'user_name': '애플티', 'review': '친절하시긴 했는데 맛은 그냥 그랬어요 ㅠㅠ'}, '5': {'user_name': 'Marco', 'review': '분위기  좋고 맛있고!'}, '6': {'user_name': '해빠라기', 'review': '진짜 맛있어유!!!!'}, '7': {'user_name': '공보의', 'review': '한라산 여행에서 우연히 들렀다가 인생 맛집 찾고 

In [ ]:
len(total_dict), len(check)

(1500, 1500)

In [ ]:
total_dict['8500']

{'rating': 0.0,
 'rating_count': 0,
 'review': {},
 'image_url': '',
 'facility': []}

In [ ]:
import pickle
with open('dict_8501_.pickle', 'rb') as fr:
    temp_dict = pickle.load(fr)

with open('check_8501_.pickle', 'rb') as fr:
    temp_check = pickle.load(fr)

In [ ]:
len(temp_dict), len(temp_check)

(1500, 1500)

In [ ]:
temp_dict["8500"], temp_check[-1]

({'rating': 4.4,
  'rating_count': 18,
  'review': {'1': {'user_name': '겨울바다', 'review': '역시 고기는 숫불에 구워야 제맛입니다.'},
   '2': {'user_name': 'Praise', 'review': '그냥 그저 그래요'},
   '3': {'user_name': 'hammi.see',
    'review': '소형 노견과 방문할거라 몇군데 고기집 전화돌리다 거절당하고 , 양반갈비는 크고 깔끔한 곳이니 더 안되겠지? 했지만 혹시나하고 도전. 짖지않고 조용하다하니 와도 된다하셔서 방문했어요 ㅎㅎ  개모차나 케이지 모두 된다 하셨는데, 들어가려니 옆 테이블 어린이 손님이 강아지 무섭다고 힝 ㅜㅜ 글서 나가야하나 했는데  2층이 비었다며 아예 통째로 내주셨어요..... 사장님 스케일 무엇?? 대관 낸 기분인것도 황송한데 고기맛은 더 황송ㅠㅠ 여행동안 백돼지 삼겹살, 흑돼지 오겹살 다 먹고 생갈비 먹으러 간건데 젤 맛있었어요. 강아지 뼈주는거 보시곤 추가 생갈비는 뼈 크고 좋은걸로 주심..... 감동의 도가니탕. 흑흑 ㅠㅠ 서비스 김찌도 몸국도 다 맛있었고 쌈채소도 신선! 큰 홀을 우리 때문에 불켜고 오르락내리락 하시는게 넘 죄송했는데  전혀 안그래도 된다고 부담되지 않게 해주시고. 사장님 어무니도 강아지 보러 왔다며 오셔서, 부족한거 없냐며 맛있게 먹고 가라고 해주셔서 감사했어요 ㅜㅜ 애견동반 되는데 맛도, 친절함까지 있는곳은 사랑입니다^^'},
   '4': {'user_name': '가족', 'review': '가격도 좋고 맛도 좋고 제주 올때마다 들리는 돼지갈비 맛집'},
   '5': {'user_name': '퍼머거',
    'review': '양도 많고. 든든하게 먹었습니다. 직원들 모두 친절해서 편하게 식사했습니다'},
   '6': {'user_name': '사람',
    'review': '홀서빙 하시는 직원분까지 아주 친절하고 고기도 맛있지만 함께 나오는 채소나 

#### json 저장

In [ ]:
pd.DataFrame(total_dict).T.head(5)  # dict to DataFrame : 문제없으며

In [ ]:
import json

total_json = json.dumps(total_dict, ensure_ascii = False)

In [ ]:
print(total_json)